# Part - 2: COVID-19 Time Series Analysis and Prediction using ML.Net framework

## COVID-19
- As per [Wiki](https://en.wikipedia.org/wiki/Coronavirus_disease_2019) **Coronavirus disease 2019** (**COVID-19**) is an infectious disease caused by severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2). The disease was first identified in 2019 in Wuhan, the capital of China's Hubei province, and has since spread globally, resulting in the ongoing 2019–20 coronavirus pandemic.
- The virus had caused a pandemic across the globe and spreading/affecting most of the nations. 
- The purpose of notebook is to visualize the trends of virus spread in various countries and explore features present in ML.Net such as DataFrame.

### Acknowledgement
- [Johns Hopkins CSSE](https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data) for dataset
- [COVID-19 data visualization](https://www.kaggle.com/akshaysb/covid-19-data-visualization) by Akshay Sb

### Dataset

- [2019 Novel Coronavirus COVID-19 (2019-nCoV) Data Repository by Johns Hopkins CSSE - Time Series](https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_time_series).

### Introduction 

This is **Part-2** of our analysis on the COVID-19 dataset provided by Johns Hopkins CSSE. In [**Part-1**](https://github.com/praveenraghuvanshi1512/TechnicalSessions/tree/31052020-virtualmlnet/31052020-virtualmlnet/src/part-1), I did data analysis on the dataset and created some tables and plots for getting insights from it. In Part-2, I'll focus on applying machine learning for making a prediction using time-series API's provided by ML.Net framework. I'll be building a model from scratch on the number of confirmed cases and predicting for the next 7 days. Later on, I'll plot these numbers for better visualization.

[**ML.Net**](https://dotnet.microsoft.com/apps/machinelearning-ai/ml-dotnet) is a cross-platform framework from Microsoft for developing Machine learning models in the .Net ecosystem. It allows .Net developers to solve business problems using machine learning algorithms leveraging their preferred language such as C#/F#. It's highly scalable and used within Microsoft in many of its products such as Bing, Powerpoint, etc.

**Disclaimer**: This is an exercise to explore different features present in ML.Net. The actual and predicted numbers might vary due to several factors such as size and features in a dataset.

### Summary

Below is the summary of steps we'll be performing

1. Define application level items
    - Nuget packages
    - Namespaces
    - Constants
     
2. Utility Functions
    - Formatters    

3. Dataset and Transformations
    - Actual from [Johns Hopkins CSSE](https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_time_series)
    - Transformed [time_series_covid19_confirmed_global_transposed.csv](time_series_covid19_confirmed_global_transposed.csv)
    
4. Data Classes
    - ConfirmedData : Provides a map between columns in a dataset
    - ConfirmedForecast : Holds predicted values

5. Data Analysis
    - Visualize Data using DataFrame API
    - Display Top 10 Rows - dataframe.Head(10)
    - Display Last 10 Rows - dataframe.Tail(10)
    - Display Dataset Statistics - dataframe.Description()
    - Plot of TotalConfimed cases vs Date

6. Load Data - MLContext
7. ML Pipeline
8. Train Model
9. Prediction/Forecasting
10. Prediction Visualization
11. Prediction Analysis
12. Conclusion

**Note** : Graphs/Plots may not render in GitHub due to secutiry reasons, however if you run this notebook locally/binder they will render.

In [1]:
#!about

The below script needs to be able to find the current output cell; this is an easy method to get it.

,.NET Interactive© 2020 Microsoft CorporationVersion: 1.0.126302+9e8f007038a86132f72a94889d805be2a14addf2Build date: 2020-05-28T18:11:57.9669641Zhttps://github.com/dotnet/interactive


### 1. Define Application wide Items

#### Nuget Packages


In [3]:
// ML.NET Nuget packages installation
#r "nuget:Microsoft.ML"
#r "nuget:Microsoft.ML.TimeSeries"
#r "nuget:Microsoft.Data.Analysis"

// Install XPlot package
#r "nuget:XPlot.Plotly"

#### Namespaces

In [4]:
using System;
using System.Collections.Generic;
using System.Linq;
using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.Data.Analysis;
using Microsoft.ML.Transforms.TimeSeries;
using Microsoft.AspNetCore.Html;
using XPlot.Plotly;

#### Constants

In [5]:
const string CONFIRMED_DATASET_FILE = "time_series_covid19_confirmed_global_transposed.csv";

// Forecast API
const int WINDOW_SIZE = 5;
const int SERIES_LENGTH = 10;
const int TRAIN_SIZE = 100;
const int HORIZON = 7;

// Dataset
const int DEFAULT_ROW_COUNT = 10;
const string TOTAL_CONFIRMED_COLUMN = "TotalConfirmed";
const string DATE_COLUMN = "Date";

### 2. Utility Functions - TBR

#### Formatters

By default the output of DataFrame is not proper and in order to display it as a table, we need to have a custom formatter implemented as shown in next cell. 

In [6]:
Formatter<DataFrame>.Register((df, writer) =>
{
    var headers = new List<IHtmlContent>();
    headers.Add(th(i("index")));
    headers.AddRange(df.Columns.Select(c => (IHtmlContent) th(c.Name)));
    var rows = new List<List<IHtmlContent>>();
    var take = DEFAULT_ROW_COUNT;
    for (var i = 0; i < Math.Min(take, df.Rows.Count); i++)
    {
        var cells = new List<IHtmlContent>();
        cells.Add(td(i));
        foreach (var obj in df.Rows[i])
        {
            cells.Add(td(obj));
        }
        rows.Add(cells);
    }

    var t = table(
        thead(
            headers),
        tbody(
            rows.Select(
                r => tr(r))));

    writer.Write(t);
}, "text/html");

### 3. Dataset and Transformations

#### Download Dataset
- Actual Dataset: [Johns Hopkins CSSE](https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_time_series)
- Transformed Dataset: [time_series_covid19_confirmed_global_transposed.csv](time_series_covid19_confirmed_global_transposed.csv)


I'll be using COVID-19 time series dataset from [Johns Hopkins CSSE](https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_time_series) and will be performing predictions using **time_series_covid19_confirmed_global.csv** file.

The data present in these files have name of the countries as Rows and dates as columns which makes it difficult to map to our classes while loading data from csv. Also, it contains data per country wise. In order to keep things simple I'll work with global count of COVID-19 cases and not specific country.

I have done few transformations to the dataset as below and created transformed csv's
- Sum cases from all the countries for a specific date
- Just have two rows with Date and Total 
- Applied transformation to the csv for converting Rows into Columns and vice-versa. [Refer](https://support.office.com/en-us/article/transpose-rotate-data-from-rows-to-columns-or-vice-versa-3419f2e3-beab-4318-aae5-d0f862209744) for transformation.
- Below transposed files have been saved in the current github directory. There is no change in dataset. The files have data till 05-27-2020
    - [time_series_covid19_confirmed_global_transposed.csv](time_series_covid19_confirmed_global_transposed.csv) : Columns - **Date, TotalConfirmed**

##### Before transformation

<img src=".\assets\time-series-before-transformation.png" alt="Time Series data before transofmation" style="zoom: 80%;" />

#### After transformation

<img src=".\assets\time-series-after-transformation.png" alt="Time Series data after transofmation" style="zoom: 80%;" />

### 4. Data Classes

Now, we need to create few data structures to map to columns within our dataset.

#### Confirmed cases

In [7]:
/// <summary>
/// Represent data for confirmed cases with a mapping to columns in a dataset
/// </summary>
public class ConfirmedData
{
    /// <summary>
    /// Date of confirmed case
    /// </summary>
    [LoadColumn(0)]
    public DateTime Date;

    /// <summary>
    /// Total no of confirmed cases on a particular date
    /// </summary>
    [LoadColumn(1)]
    public float TotalConfirmed;
}

In [8]:
/// <summary>
/// Prediction/Forecast for Confirmed cases
/// </summary>
internal class ConfirmedForecast
{
    /// <summary>
    /// No of predicted confirmed cases for multiple days
    /// </summary>
    public float[] Forecast { get; set; }
}

### 5. Data Analysis

For loading data from csv, first we need to create MLContext that acts as a starting point for creating a machine learning model in ML.Net. Few things to note
- Set hasHeader as true as our dataset has header
- Add separatorChar to ',' as its a csv

#### Visualize Data - DataFrame

In [9]:
var predictedDf = DataFrame.LoadCsv(CONFIRMED_DATASET_FILE);

In [10]:
predictedDf.Head(DEFAULT_ROW_COUNT)

0,1/22/2020,555
1,1/23/2020,654
2,1/24/2020,941
3,1/25/2020,1434
4,1/26/2020,2118
5,1/27/2020,2927
6,1/28/2020,5578
7,1/29/2020,6166
8,1/30/2020,8234
9,1/31/2020,9927


In [11]:
predictedDf.Tail(DEFAULT_ROW_COUNT)

0,5/18/2020,4801943
1,5/19/2020,4897492
2,5/20/2020,4996472
3,5/21/2020,5102424
4,5/22/2020,5211156
5,5/23/2020,5311020
6,5/24/2020,5407613
7,5/25/2020,5495061
8,5/26/2020,5589626
9,5/27/2020,5691790


In [12]:
predictedDf.Description()

0,Length (excluding null values),127
1,Max,5691790
2,Min,555
3,Mean,1531269


##### Number of Confirmed cases over Time

In [13]:
// Number of confirmed cases over time
var totalConfirmedDateColumn = predictedDf.Columns[DATE_COLUMN];
var totalConfirmedColumn = predictedDf.Columns[TOTAL_CONFIRMED_COLUMN];

var dates = new List<string>();
var totalConfirmedCases = new List<string>();
for (int index = 0; index < totalConfirmedDateColumn.Length; index++)
{
    dates.Add(totalConfirmedDateColumn[index].ToString());
    totalConfirmedCases.Add(totalConfirmedColumn[index].ToString());
}

In [14]:
var title = "Number of Confirmed Cases over Time";
var confirmedTimeGraph = new Graph.Scattergl()
    {
        x = dates.ToArray(),
        y = totalConfirmedCases.ToArray(),
        mode = "lines+markers"
    };
    


var chart = Chart.Plot(confirmedTimeGraph);
chart.WithTitle(title);
display(chart);

**Analysis**
- Duration: 1/22/2020 through 5/27/2020
- Total records: 127
- Case on first day: 555
- Case on last day: 5691790
- No of confirmed cases was low in the beginning, there was first jump around 2/12/2020 and an exponential jump around 3/22/2020.
- Cases have been increasing at an alarming rate in the past two months.

### 6. Load Data - MLContext

In [15]:
var context = new MLContext();

In [16]:
var data = context.Data.LoadFromTextFile<ConfirmedData>(CONFIRMED_DATASET_FILE, hasHeader: true, separatorChar: ',');

### 7. ML Pipeline

For creating ML Pipeline for a time-series analysis, we'll use [Single Spectrum Analysis](https://en.wikipedia.org/wiki/Singular_spectrum_analysis). ML.Net provides built in API for same, more details could be found at [TimeSeriesCatalog.ForecastBySsa](https://docs.microsoft.com/en-us/dotnet/api/microsoft.ml.timeseriescatalog.forecastbyssa?view=ml-dotnet) 

In [17]:
var pipeline = context.Forecasting.ForecastBySsa(
                nameof(ConfirmedForecast.Forecast),
                nameof(ConfirmedData.TotalConfirmed),
                WINDOW_SIZE, 
                SERIES_LENGTH,
                TRAIN_SIZE,
                HORIZON);

In [18]:
#!log

Logging enabled

### 8. Train Model

We are ready with our pipeline and ready to train the model

In [19]:
var model = pipeline.Fit(data);

SubmitCode: var model = pipeline.Fit(data);

CodeSubmissionReceived: var model = pipeline.Fit(data);

CompleteCodeSubmissionReceived: var model = pipeline.Fit(data);

2020-05-29T06:06:38.3988147Z [00-af5397e93ec0db4ca70e3467b33039b8-a5081a9355365549-00]  ℹ OnAssemblyLoad: C:\Users\praghuvanshi\.nuget\packages\microsoft.ml.cpumath\1.5.0\lib\netcoreapp3.1\Microsoft.ML.CpuMath.dll 

2020-05-29T06:06:38.4122851Z [00-af5397e93ec0db4ca70e3467b33039b8-a5081a9355365549-00]  ℹ OnAssemblyLoad: C:\Users\praghuvanshi\.nuget\packages\microsoft.ml.cpumath\1.5.0\lib\netcoreapp3.1\Microsoft.ML.CpuMath.dll 

2020-05-29T06:06:38.4179725Z [00-af5397e93ec0db4ca70e3467b33039b8-a5081a9355365549-00]  ℹ OnAssemblyLoad: C:\Program Files\dotnet\shared\Microsoft.NETCore.App\3.1.4\System.Runtime.Intrinsics.dll 

2020-05-29T06:06:38.4182246Z [00-af5397e93ec0db4ca70e3467b33039b8-a5081a9355365549-00]  ℹ OnAssemblyLoad: C:\Program Files\dotnet\shared\Microsoft.NETCore.App\3.1.4\System.Runtime.Intrinsics.dll 

Unhandled exception: System.DllNotFoundException: Unable to load DLL 'MklImports' or one of its dependencies: The specified module could not be found. (0x8007007E)
   at Microsoft.ML.Transforms.TimeSeries.EigenUtils.Dsytrd(Layout matrixLayout, Uplo uplo, Int32 n, Double[] a, Int32 lda, Double[] d, Double[] e, Double[] tau)
   at Microsoft.ML.Transforms.TimeSeries.EigenUtils.MklSymmetricEigenDecomposition(Single[] input, Int32 size, Single[]& eigenValues, Single[]& eigenVectors)
   at Microsoft.ML.Transforms.TimeSeries.TrajectoryMatrix.ComputeSvd(Single[]& singularValues, Single[]& leftSingularvectors)
   at Microsoft.ML.Transforms.TimeSeries.AdaptiveSingularSpectrumSequenceModelerInternal.TrainCore(Single[] dataArray, Int32 originalSeriesLength)
   at Microsoft.ML.Transforms.TimeSeries.AdaptiveSingularSpectrumSequenceModelerInternal.Train(RoleMappedData data)
   at Microsoft.ML.Transforms.TimeSeries.SsaForecastingTransformer..ctor(IHostEnvironment env, Options options, IDataView 

CommandFailed: System.DllNotFoundException: Unable to load DLL 'MklImports' or one of its dependencies: The specified module could not be found. (0x8007007E)
   at Microsoft.ML.Transforms.TimeSeries.EigenUtils.Dsytrd(Layout matrixLayout, Uplo uplo, Int32 n, Double[] a, Int32 lda, Double[] d, Double[] e, Double[] tau)
   at Microsoft.ML.Transforms.TimeSeries.EigenUtils.MklSymmetricEigenDecomposition(Single[] input, Int32 size, Single[]& eigenValues, Single[]& eigenVectors)
   at Microsoft.ML.Transforms.TimeSeries.TrajectoryMatrix.ComputeSvd(Single[]& singularValues, Single[]& leftSingularvectors)
   at Microsoft.ML.Transforms.TimeSeries.AdaptiveSingularSpectrumSequenceModelerInternal.TrainCore(Single[] dataArray, Int32 originalSeriesLength)
   at Microsoft.ML.Transforms.TimeSeries.AdaptiveSingularSpectrumSequenceModelerInternal.Train(RoleMappedData data)
   at Microsoft.ML.Transforms.TimeSeries.SsaForecastingTransformer..ctor(IHostEnvironment env, Options options, IDataView input)
   at

### 9. Prediction/Forecasting - 7 days

Our model is trained and we need to do prediction for next 7(Horizon) days.
Time-series provides its own engine for making prediction which is similar to PredictionEngine present in ML.Net. Predicted values show an increasing trend which is in alignment with recent past values.

In [18]:
var forecastingEngine = model.CreateTimeSeriesEngine<ConfirmedData, ConfirmedForecast>(context);
var forecasts = forecastingEngine.Predict();
display(forecasts.Forecast.Select(x => (int) x))

SubmitCode: var forecastingEngine = model.CreateTimeSeriesEngi ...

CodeSubmissionReceived: var forecastingEngine = model.CreateTimeSeriesEngi ...

CompleteCodeSubmissionReceived: var forecastingEngine = model.CreateTimeSeriesEngi ...

Unhandled exception: System.ArgumentNullException: Value cannot be null. (Parameter 'transformer')
   at Microsoft.ML.Runtime.Contracts.CheckValue[T](IExceptionContext ctx, T val, String paramName)
   at Microsoft.ML.Transforms.TimeSeries.PredictionFunctionExtensions.CreateTimeSeriesEngine[TSrc,TDst](ITransformer transformer, IHostEnvironment env, Boolean ignoreMissingColumns, SchemaDefinition inputSchemaDefinition, SchemaDefinition outputSchemaDefinition)
   at Submission#24.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location where exception was thrown ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

CommandFailed: System.ArgumentNullException: Value cannot be null. (Parameter 'transformer')
   at Microsoft.ML.Runtime.Contracts.CheckValue[T](IExceptionContext ctx, T val, String paramName)
   at Microsoft.ML.Transforms.TimeSeries.PredictionFunctionExtensions.CreateTimeSeriesEngine[TSrc,TDst](ITransformer transformer, IHostEnvironment env, Boolean ignoreMissingColumns, SchemaDefinition inputSchemaDefinition, SchemaDefinition outputSchemaDefinition)
   at Submission#24.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location where exception was thrown ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

### 10. Prediction Visualization

In [19]:
var lastDate = DateTime.Parse(dates.LastOrDefault());
var predictionStartDate = lastDate.AddDays(1);

for (int index = 0; index < HORIZON; index++)
{
    dates.Add(lastDate.AddDays(index + 1).ToShortDateString());
    totalConfirmedCases.Add(forecasts.Forecast[index].ToString());
}

SubmitCode: var lastDate = DateTime.Parse(dates.LastOrDefault( ...

CodeSubmissionReceived: var lastDate = DateTime.Parse(dates.LastOrDefault( ...

CompleteCodeSubmissionReceived: var lastDate = DateTime.Parse(dates.LastOrDefault( ...

Unhandled exception: System.NullReferenceException: Object reference not set to an instance of an object.
   at Submission#25.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location where exception was thrown ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

CommandFailed: System.NullReferenceException: Object reference not set to an instance of an object.
   at Submission#25.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location where exception was thrown ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

In [20]:
var title = "Number of Confirmed Cases over Time";
var layout = new Layout.Layout();
layout.shapes = new List<Graph.Shape>
{
    new Graph.Shape
    {
        x0 = predictionStartDate.ToShortDateString(),
        x1 = predictionStartDate.ToShortDateString(),
        y0 = "0",
        y1 = "1",
        xref = 'x',
        yref = "paper",
        line = new Graph.Line() {color = "red", width = 2}
    }
};

var chart1 = Chart.Plot(
new [] 
    {
        new Graph.Scattergl()
        {
            x = dates.ToArray(),
            y = totalConfirmedCases.ToArray(),
            mode = "lines+markers"
        }
    },
    layout
);

chart1.WithTitle(title);
display(chart1);

SubmitCode: var title = "Number of Confirmed Cases over Time"; ...

CodeSubmissionReceived: var title = "Number of Confirmed Cases over Time"; ...

CompleteCodeSubmissionReceived: var title = "Number of Confirmed Cases over Time"; ...

### 11. Analysis

Comparing the plots before and after prediction, it seems our ML model has performed reasonably well. The red line represents the data on future date(5/8/2020). Beyond this, we predicted for 7 days. Looking at the plot, there is a sudden drop on 5/8/2020 which could be accounted due to insufficient data as we have only 107 records. However we see an increasing trend for next 7 days in alignment with previous confirmed cases. We can extend this model for predicting confirmed cases for any number of days by changing HORIZON constant value. This plot is helpful in analysing the increased number of cases and allow authorities to take preventive actions to lower the number.

## Conclusion

I hope you have enjoyed reading the notebook, and might have got some idea on the powerful framework ML.Net. ML.Net is a very fast emerging framework for .Net developers which abstracts lot of complexity present in the field of Data science and Machine Learning. The focus of Part-2 notebook is leverage ML.Net for making predictions using time-series API. The model generated can be saved as a zip file and used in different applications.

Feedback/Suggestion are welcome. Please reach out to me through below channels

**Contact**

**Email    :** praveenraghuvanshi@gmail.com  
**LinkedIn :** https://in.linkedin.com/in/praveenraghuvanshi  
**Github   :** https://github.com/praveenraghuvanshi1512  
**Twitter  :** @praveenraghuvan



## References
- [Tutorial: Forecast bike rental service demand with time series analysis and ML.NET](https://docs.microsoft.com/en-us/dotnet/machine-learning/tutorials/time-series-demand-forecasting#evaluate-the-model)
- [Time Series Forecasting in ML.NET and Azure ML notebooks](https://github.com/gvashishtha/time-series-mlnet/blob/master/time-series-forecast.ipynb) by Gopal Vashishtha

#  ******************** Be Safe **********************